In [1]:
from twython import Twython as tw
from twython import TwythonError

In [2]:
# App keys and secrets stored as environment variables
import os
APP_KEY = os.environ.get('TWITTER_APP_KEY')
APP_SECRET = os.environ.get('TWITTER_APP_SECRET')

Access_Token = os.environ.get('TWITTER_ACCESS_TOKEN')
Access_Secret = os.environ.get('TWITTER_ACCESS_SECRET')

twitter = tw(APP_KEY, APP_SECRET, Access_Token, Access_Secret)

In [3]:
import pandas as pd

In [4]:
# Define a query

bitcoin_query = {'q': 'bitcoin',  
        'result_type': 'popular',
        'count': 100,
        'lang': 'en',
        }

In [5]:
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'user_ID': []}  
for status in twitter.search(**bitcoin_query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['user_ID'].append(status['user'])

In [6]:
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  

In [49]:
# Store list for messing around with later
import time
timestr = time.strftime("%Y%m%d")
filename = 'tweets-'+timestr

df.to_csv(os.path.join(path,filename))

In [7]:
# Remove duplicates to find individual users

df = df.drop_duplicates(subset='user', keep='first', inplace=False)
df.head()

,date,favorite_count,text,user,user_ID
1,Mon Dec 10 21:13:44 +0000 2018,2229,The banks want Bitcoin to fail.\n\nNever forge...,APompliano,"{'id': 339061487, 'id_str': '339061487', 'name..."
3,Mon Dec 10 21:45:10 +0000 2018,525,You will never fully understand the power and ...,getongab,"{'id': 767960339547774976, 'id_str': '76796033..."
2,Sun Dec 09 19:59:01 +0000 2018,505,"Just published: ""Bitcoin &amp; Ethereum: Price...",cburniske,"{'id': 710463119886262272, 'id_str': '71046311..."
5,Mon Dec 10 15:50:14 +0000 2018,182,"“Fundamentally, for this thing to work, it can...",TDANetwork,"{'id': 872519322077470721, 'id_str': '87251932..."
7,Mon Dec 10 11:24:56 +0000 2018,174,Bitcoin still has the potential to drop toward...,coindesk,"{'id': 1333467482, 'id_str': '1333467482', 'na..."


In [8]:
# Pull user names & ID's and store

influencers = df.loc[:, 'user']

In [9]:
# Storing the newly found users in a list. 

my_user_id = 705469309
my_list_name = 'tweet_smart_list'
# Create dictionary of my lists
my_lists_dict = twitter.show_owned_lists(user_id = my_user_id) 
#Split so that running code doesn't keep sending requests to Twitter. 

In [10]:
my_lists_df = pd.DataFrame(my_lists_dict['lists']) 
names = my_lists_df.loc[my_lists_df.name == my_list_name]
my_list_id = int(names.id)
list_slug = names.slug

In [11]:
for user in influencers:
    try:
        twitter.add_list_member(list_id = my_list_id, screen_name=user)
    except TwythonError as e:
        print(e)

        